The aim of this notebook is to express the semantics of schema evolution of parquet files using both Spark and parquet2hive + Presto. Note that all parquet2hive is doing is reading the schema *from the most recently created file*, so in some cases this could be changed without changing Presto's underlying facilities by reading *all* files.

In this notebook I am running Spark locally, and using a remote Presto cluster. To connect to this cluster I'm using parquet2hive_server [0], which is just a simple API for parquet2hive on the remote cluster. To run this notebook successfully, you'll need to run the following on the Presto cluster:

```
sudo pip install parquet2hive_server
start_parquet2hive_server
```

[0] http://www.github.com/fbertsch/parquet2hive_server

In [1]:
from parquet2hive_server.client import Parquet2HiveClient
from pyhive import presto
from pprint import pprint

presto_dns = 'ec2-54-149-100-125.us-west-2.compute.amazonaws.com'

client = Parquet2HiveClient(presto_dns + ':5129')

In [2]:
import boto3

bucket, prefix = "telemetry-test-bucket", "schema_evolution"
dataset = "s3://{}/{}/".format(bucket, prefix)
partition = '/type='

s3 = boto3.resource('s3')
objects_to_delete = s3.meta.client.list_objects(Bucket=bucket, Prefix=prefix)

delete_keys = {}
delete_keys['Objects'] = [{'Key' : k} for k in [obj['Key'] for obj in objects_to_delete.get('Contents', [])]]

try:
    _ = s3.meta.client.delete_objects(Bucket=bucket, Delete=delete_keys)
except Exception:
    pass

In [3]:
conn = presto.connect(host=presto_dns, port='8889')
cursor = conn.cursor()

def get_schema(_cursor, _v):
    """Prints the schema in a similar format to spark's dataframe.printSchema()"""
    _cursor.execute('describe schema_evolution_{}'.format(_v))
    return '\\\n'.join(['root'] + [' |-- {}: {}'.format(a, b) for a, b, _, _ in _cursor.fetchall()])

def execute(_cursor, _query):
    _cursor.execute(_query)
    results = _cursor.fetchall()
    colnames = [c[0] for c in _cursor.description]
    return '\\\n'.join([', '.join(['{}={}'.format(c,r) for c,r in zip(colnames, res)]) for res in results])

## Adding a Column

In [4]:
v = 'v1'

In [5]:
rdd = sc.parallelize([(0,),(1,)], 1)
df = sqlContext.createDataFrame(rdd, ['id'])
df.write.parquet(dataset + v + partition + '1')

In [6]:
rdd = sc.parallelize([(0,0),(1,1)], 1)
df = sqlContext.createDataFrame(rdd, ['id', 'score'])
df.write.parquet(dataset + v + partition  + '2')

### Spark

In [7]:
sqlContext.read.load(dataset + v, 'parquet').printSchema()

root
 |-- id: long (nullable = true)
 |-- type: string (nullable = true)



In [8]:
sqlContext.read.load(dataset + v, 'parquet').collect()

[Row(id=0, type=u'2'),
 Row(id=1, type=u'2'),
 Row(id=0, type=u'1'),
 Row(id=1, type=u'1')]

In [9]:
sqlContext.read.option("mergeSchema", "true").load(dataset + v, 'parquet').printSchema()

root
 |-- id: long (nullable = true)
 |-- score: long (nullable = true)
 |-- type: string (nullable = true)



In [10]:
sqlContext.read.option("mergeSchema", "true").load(dataset + v, 'parquet').collect()

[Row(id=0, score=0, type=u'2'),
 Row(id=1, score=1, type=u'2'),
 Row(id=0, score=None, type=u'1'),
 Row(id=1, score=None, type=u'1')]

### Presto

In [11]:
client.load(dataset=dataset, dv=v)

(200, u'{"Result": [null, null]}\n')

In [12]:
print get_schema(cursor, v)

root\
 |-- id: bigint\
 |-- score: bigint\
 |-- type: varchar


In [13]:
print execute(cursor, "SELECT * FROM schema_evolution_{}".format(v))

id=0, score=0, type=2\
id=1, score=1, type=2\
id=0, score=None, type=1\
id=1, score=None, type=1


## Removing a Column

In [14]:
v = 'v2'

rdd = sc.parallelize([(0,0),(1,1)], 1)
df = sqlContext.createDataFrame(rdd, ['id', 'score'])
df.write.parquet(dataset + v + partition  + '1')

In [15]:
rdd = sc.parallelize([(0,),(1,)], 1)
df = sqlContext.createDataFrame(rdd, ['id'])
df.write.parquet(dataset + v + partition + '2')

### Spark

In [16]:
sqlContext.read.load(dataset + v, 'parquet').printSchema()

root
 |-- id: long (nullable = true)
 |-- score: long (nullable = true)
 |-- type: string (nullable = true)



In [17]:
sqlContext.read.load(dataset + v, 'parquet').collect()

[Row(id=0, score=0, type=u'1'),
 Row(id=1, score=1, type=u'1'),
 Row(id=0, score=None, type=u'2'),
 Row(id=1, score=None, type=u'2')]

In [18]:
sqlContext.read.option("mergeSchema", "true").load(dataset + v, 'parquet').printSchema()

root
 |-- id: long (nullable = true)
 |-- score: long (nullable = true)
 |-- type: string (nullable = true)



In [19]:
sqlContext.read.option("mergeSchema", "true").load(dataset + v, 'parquet').collect()

[Row(id=0, score=0, type=u'1'),
 Row(id=1, score=1, type=u'1'),
 Row(id=0, score=None, type=u'2'),
 Row(id=1, score=None, type=u'2')]

### Presto

In [20]:
client.load(dataset=dataset, dv=v)

(200, u'{"Result": [null, null]}\n')

In [21]:
print get_schema(cursor, v)

root\
 |-- id: bigint\
 |-- type: varchar


In [22]:
print execute(cursor, "SELECT * FROM schema_evolution_{}".format(v))

id=0, type=1\
id=1, type=1\
id=0, type=2\
id=1, type=2


## Renaming a Column

In [23]:
v = 'v3'

In [24]:
rdd = sc.parallelize([(0,),(1,)], 1)
df = sqlContext.createDataFrame(rdd, ['id'])
df.write.parquet(dataset + v + partition + '1')

In [25]:
rdd = sc.parallelize([(0,),(1,)], 1)
df = sqlContext.createDataFrame(rdd, ['score'])
df.write.parquet(dataset + v + partition + '2')

### Spark

In [26]:
sqlContext.read.load(dataset + v, 'parquet').printSchema()

root
 |-- id: long (nullable = true)
 |-- type: string (nullable = true)



In [27]:
sqlContext.read.load(dataset + v, 'parquet').collect()

[Row(id=None, type=u'2'),
 Row(id=None, type=u'2'),
 Row(id=0, type=u'1'),
 Row(id=1, type=u'1')]

In [28]:
sqlContext.read.option("mergeSchema", "true").load(dataset + v, 'parquet').printSchema()

root
 |-- id: long (nullable = true)
 |-- score: long (nullable = true)
 |-- type: string (nullable = true)



In [29]:
sqlContext.read.option("mergeSchema", "true").load(dataset + v, 'parquet').collect()

[Row(id=None, score=0, type=u'2'),
 Row(id=None, score=1, type=u'2'),
 Row(id=0, score=None, type=u'1'),
 Row(id=1, score=None, type=u'1')]

### Presto

In [30]:
client.load(dataset=dataset, dv=v)

(200, u'{"Result": [null, null]}\n')

In [31]:
print get_schema(cursor, v)

root\
 |-- score: bigint\
 |-- type: varchar


In [32]:
print execute(cursor, "SELECT * FROM schema_evolution_{}".format(v))

score=0, type=2\
score=1, type=2\
score=0, type=1\
score=1, type=1


### Replace Column

Note that this is similar to "rename column", but the new data has a different type.

In [33]:
v = 'v4'

rdd = sc.parallelize([(0,),(1,)], 1)
df = sqlContext.createDataFrame(rdd, ['id'])
df.write.parquet(dataset + v + partition + '1')

In [34]:
rdd = sc.parallelize([('a',),('b',)], 1)
df = sqlContext.createDataFrame(rdd, ['score'])
df.write.parquet(dataset + v + partition + '2')

### Spark

In [35]:
sqlContext.read.load(dataset + v, 'parquet').printSchema()

root
 |-- id: long (nullable = true)
 |-- type: string (nullable = true)



In [36]:
df = sqlContext.read.load(dataset + v, 'parquet').collect()

In [37]:
sqlContext.read.option("mergeSchema", "true").load(dataset + v, 'parquet').printSchema()

root
 |-- id: long (nullable = true)
 |-- score: string (nullable = true)
 |-- type: string (nullable = true)



In [38]:
sqlContext.read.option("mergeSchema", "true").load(dataset + v, 'parquet').collect()

[Row(id=0, score=None, type=u'1'),
 Row(id=1, score=None, type=u'1'),
 Row(id=None, score=u'a', type=u'2'),
 Row(id=None, score=u'b', type=u'2')]

### Presto

In [39]:
client.load(dataset=dataset, dv=v)

(200, u'{"Result": [null, null]}\n')

In [40]:
print get_schema(cursor, v)

root\
 |-- score: varchar\
 |-- type: varchar


In [41]:
print execute(cursor, "SELECT * FROM schema_evolution_{}".format(v))

DatabaseError: {u'errorCode': 65536, u'message': u'com.facebook.presto.spi.type.VarcharType', u'errorType': u'INTERNAL_ERROR', u'failureInfo': {u'suppressed': [], u'message': u'com.facebook.presto.spi.type.VarcharType', u'type': u'java.lang.UnsupportedOperationException', u'stack': [u'com.facebook.presto.spi.type.AbstractType.writeLong(AbstractType.java:111)', u'com.facebook.presto.hive.parquet.reader.ParquetLongColumnReader.readValue(ParquetLongColumnReader.java:32)', u'com.facebook.presto.hive.parquet.reader.ParquetColumnReader.readValues(ParquetColumnReader.java:173)', u'com.facebook.presto.hive.parquet.reader.ParquetColumnReader.readPrimitive(ParquetColumnReader.java:155)', u'com.facebook.presto.hive.parquet.reader.ParquetReader.readPrimitive(ParquetReader.java:239)', u'com.facebook.presto.hive.parquet.reader.ParquetReader.readPrimitive(ParquetReader.java:221)', u'com.facebook.presto.hive.parquet.ParquetPageSource$ParquetBlockLoader.load(ParquetPageSource.java:283)', u'com.facebook

## Tranpose Columns

In [42]:
v = 'v5'

rdd = sc.parallelize([(0,'a','b')], 1)
df = sqlContext.createDataFrame(rdd, ['id', 'transpose_a', 'transpose_b'])
df.write.parquet(dataset + v + partition + '1')

In [43]:
rdd = sc.parallelize([(1,'b','a')], 1)
df = sqlContext.createDataFrame(rdd, ['id', 'transpose_b', 'transpose_a'])
df.write.parquet(dataset + v + partition + '2')

### Spark

In [44]:
sqlContext.read.load(dataset + v, 'parquet').printSchema()

root
 |-- id: long (nullable = true)
 |-- transpose_a: string (nullable = true)
 |-- transpose_b: string (nullable = true)
 |-- type: string (nullable = true)



In [45]:
sqlContext.read.load(dataset + v, 'parquet').select('transpose_a','transpose_b').collect()

[Row(transpose_a=u'a', transpose_b=u'b'),
 Row(transpose_a=u'a', transpose_b=u'b')]

In [46]:
sqlContext.read.option("mergeSchema", "true").load(dataset + v, 'parquet').printSchema()

root
 |-- id: long (nullable = true)
 |-- transpose_a: string (nullable = true)
 |-- transpose_b: string (nullable = true)
 |-- type: string (nullable = true)



In [47]:
sqlContext.read.option("mergeSchema", "true").load(dataset + v, 'parquet').select('transpose_a','transpose_b').collect()

[Row(transpose_a=u'a', transpose_b=u'b'),
 Row(transpose_a=u'a', transpose_b=u'b')]

### Presto

In [48]:
client.load(dataset=dataset, dv=v)

(200, u'{"Result": [null, null]}\n')

In [49]:
print(get_schema(cursor, v))

root\
 |-- id: bigint\
 |-- transpose_b: varchar\
 |-- transpose_a: varchar\
 |-- type: varchar


In [50]:
print execute(cursor, "SELECT * FROM schema_evolution_{}".format(v))

id=1, transpose_b=b, transpose_a=a, type=2\
id=0, transpose_b=a, transpose_a=b, type=1


## Transpose, Delete and Add Columns

In [51]:
v = 'v6'

rdd = sc.parallelize([(0,'r','t')], 1)
df = sqlContext.createDataFrame(rdd, ['id', 'removed', 'transposed'])
df.write.parquet(dataset + v + partition + '1')

In [52]:
rdd = sc.parallelize([(1,'t','a')], 1)
df = sqlContext.createDataFrame(rdd, ['id', 'transposed', 'added'])
df.write.parquet(dataset + v + partition + '2')

### Spark

In [53]:
sqlContext.read.load(dataset + v, 'parquet').printSchema()

root
 |-- id: long (nullable = true)
 |-- removed: string (nullable = true)
 |-- transposed: string (nullable = true)
 |-- type: string (nullable = true)



In [54]:
sqlContext.read.load(dataset + v, 'parquet').collect()

[Row(id=0, removed=u'r', transposed=u't', type=u'1'),
 Row(id=1, removed=None, transposed=u't', type=u'2')]

In [55]:
sqlContext.read.option("mergeSchema", "true").load(dataset + v, 'parquet').printSchema()

root
 |-- id: long (nullable = true)
 |-- removed: string (nullable = true)
 |-- transposed: string (nullable = true)
 |-- added: string (nullable = true)
 |-- type: string (nullable = true)



In [56]:
sqlContext.read.option("mergeSchema", "true").load(dataset + v, 'parquet').collect()

[Row(id=0, removed=u'r', transposed=u't', added=None, type=u'1'),
 Row(id=1, removed=None, transposed=u't', added=u'a', type=u'2')]

### Presto

In [57]:
client.load(dataset=dataset, dv=v)

(200, u'{"Result": [null, null]}\n')

In [58]:
print(get_schema(cursor, v))

root\
 |-- id: bigint\
 |-- transposed: varchar\
 |-- added: varchar\
 |-- type: varchar


In [59]:
print execute(cursor, "SELECT * FROM schema_evolution_{}".format(v))

id=0, transposed=r, added=t, type=1\
id=1, transposed=t, added=a, type=2


# Nested Row Type - Adding a Subcolumn

In [60]:
from pyspark.sql.types import *

v = 'v7'

df = sqlContext.createDataFrame([[[1, 'e']]], StructType([
    StructField("nested", 
                StructType([
                        StructField("id", LongType()), 
                        StructField("exists", StringType())
                    ])
               )
]))

df.write.parquet(dataset + v + partition + '1')

In [61]:
df = sqlContext.createDataFrame([[[1, 'e', 'a']]], StructType([
    StructField("nested", 
                StructType([
                        StructField("id", LongType()), 
                        StructField("exists", StringType()),
                        StructField("added", StringType())
                    ])
               )
]))
df.write.parquet(dataset + v + partition + '2')

## Spark

In [62]:
sqlContext.read.load(dataset + v, 'parquet').printSchema()

root
 |-- nested: struct (nullable = true)
 |    |-- id: long (nullable = true)
 |    |-- exists: string (nullable = true)
 |-- type: string (nullable = true)



In [63]:
sqlContext.read.load(dataset + v, 'parquet').collect()

[Row(nested=Row(id=1, exists=u'e'), type=u'2'),
 Row(nested=Row(id=1, exists=u'e'), type=u'1')]

In [64]:
sqlContext.read.option("mergeSchema", "true").load(dataset + v, 'parquet').printSchema()

root
 |-- nested: struct (nullable = true)
 |    |-- id: long (nullable = true)
 |    |-- exists: string (nullable = true)
 |    |-- added: string (nullable = true)
 |-- type: string (nullable = true)



In [65]:
sqlContext.read.option("mergeSchema", "true").load(dataset + v, 'parquet').collect()

[Row(nested=Row(id=1, exists=u'e', added=u'a'), type=u'2'),
 Row(nested=Row(id=1, exists=u'e', added=None), type=u'1')]

## Presto

In [66]:
client.load(dataset=dataset, dv=v)

(200, u'{"Result": [null, null]}\n')

In [67]:
print(get_schema(cursor, v))

root\
 |-- nested: row(id bigint, exists varchar, added varchar)\
 |-- type: varchar


In [68]:
print execute(cursor, "SELECT * FROM schema_evolution_{}".format(v))

nested=[1, u'e', u'a'], type=2\
nested=[1, u'e', None], type=1


# Nested Row Type - Removing a Subcolumn

In [69]:
v = 'v8'

df = sqlContext.createDataFrame([[[1, 'e', 'r']]], StructType([
    StructField("nested", 
                StructType([
                        StructField("id", LongType()), 
                        StructField("exists", StringType()),
                        StructField("removed", StringType())
                    ])
               )
]))

df.write.parquet(dataset + v + partition + '1')

In [70]:
df = sqlContext.createDataFrame([[[1, 'e']]], StructType([
    StructField("nested", 
                StructType([
                        StructField("id", LongType()), 
                        StructField("exists", StringType())
                    ])
               )
]))

df.write.parquet(dataset + v + partition + '2')

## Spark

In [71]:
sqlContext.read.load(dataset + v, 'parquet').printSchema()

root
 |-- nested: struct (nullable = true)
 |    |-- id: long (nullable = true)
 |    |-- exists: string (nullable = true)
 |    |-- removed: string (nullable = true)
 |-- type: string (nullable = true)



In [72]:
sqlContext.read.load(dataset + v, 'parquet').collect()

[Row(nested=Row(id=1, exists=u'e', removed=u'r'), type=u'1'),
 Row(nested=Row(id=1, exists=u'e', removed=None), type=u'2')]

In [73]:
sqlContext.read.option("mergeSchema", "true").load(dataset + v, 'parquet').printSchema()

root
 |-- nested: struct (nullable = true)
 |    |-- id: long (nullable = true)
 |    |-- exists: string (nullable = true)
 |    |-- removed: string (nullable = true)
 |-- type: string (nullable = true)



In [74]:
sqlContext.read.option("mergeSchema", "true").load(dataset + v, 'parquet').collect()

[Row(nested=Row(id=1, exists=u'e', removed=u'r'), type=u'1'),
 Row(nested=Row(id=1, exists=u'e', removed=None), type=u'2')]

## Presto

In [75]:
client.load(dataset=dataset, dv=v)

(200, u'{"Result": [null, null]}\n')

In [76]:
print(get_schema(cursor, v))

root\
 |-- nested: row(id bigint, exists varchar)\
 |-- type: varchar


In [77]:
print execute(cursor, "SELECT * FROM schema_evolution_{}".format(v))

nested=[1, u'e'], type=2\
nested=[1, u'e'], type=1
